In [23]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from datetime import  timedelta, timedelta, datetime

# Extração de dados API

## Lista de DAGS

In [24]:
baseUrl = "http://localhost:8080/api/v1"

username = 'airflow'
password = '1234'

In [25]:
try:
  response = requests.get(baseUrl + "/dags?only_active=true&tags=ativo", auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    dagsDF = pd.DataFrame(data['dags'])
    dagsDF = dagsDF[['dag_id', 'dag_display_name', 'is_active', 'is_paused', 'is_subdag', 'root_dag_id', 'owners', 'tags', 'schedule_interval', 'timetable_description']]
except Exception as e:
  print(e)

sucesso!


In [26]:
dagsDF.head()

,dag_id,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,earthquake_data_upstream,Upstream EarthQuake Data,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
1,processa_dados_EarthQuake,Data Count,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
2,teste_execucao,Teste de Execucao,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"
3,teste_falha,Teste de Falha,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"


## DAGS em Execucao...

In [27]:
today = datetime.now()
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
print(yesterday, tomorrow)

2024-10-21 11:32:30.013176 2024-10-23 11:32:30.013176


In [28]:
payload = {
    # "dag_ids": [
    #     "processa_dados_EarthQuake",
    #     "earthquake_data_upstream"
    # ],
    "states": [
        "failed",
        "running",
        "queued",
        "success"
    ],
    "start_date_gte": yesterday.strftime("%Y-%m-%dT00:00:00.000Z"),
    "start_date_lte": tomorrow.strftime("%Y-%m-%dT00:00:00.000Z")
}

In [29]:
try:
  response = requests.post(baseUrl + "/dags/~/dagRuns/list", json=payload, auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    runsDF = pd.DataFrame(data['dag_runs'])
    runsDF = runsDF[['dag_id','start_date', 'end_date', 'run_type', 'state']]
    
    runsDF['start_date'] = pd.to_datetime(runsDF['start_date'])
    runsDF['end_date'] = pd.to_datetime(runsDF['end_date'])
    runsDF['start_hour'] = runsDF['start_date'].dt.hour
    runsDF['execution_time'] = (runsDF['end_date'] - runsDF['start_date']).dt.total_seconds() / 60
    
    runsDF['start_date'] = runsDF['start_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    runsDF['end_date'] = runsDF['end_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    runsDF['execution_date'] = runsDF['execution_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    
except Exception as e:
  print(e)

runsDF.head()

sucesso!
'execution_date'


,dag_id,start_date,end_date,run_type,state,start_hour,execution_time
0,earthquake_data_upstream,22-10-2024 14:03:39,22-10-2024 14:04:11,manual,success,14,0 days 00:00:31.508536
1,teste_falha,22-10-2024 14:03:38,22-10-2024 14:10:19,manual,failed,14,0 days 00:06:40.432990
2,teste_execucao,22-10-2024 14:03:38,22-10-2024 14:07:58,manual,success,14,0 days 00:04:19.294494
3,processa_dados_EarthQuake,22-10-2024 14:03:39,22-10-2024 14:04:07,manual,success,14,0 days 00:00:27.984439
4,earthquake_data_upstream,22-10-2024 14:09:47,22-10-2024 14:10:25,manual,success,14,0 days 00:00:38.089895


## Join df

In [30]:
df = pd.merge(runsDF, dagsDF, on='dag_id', how='inner')
df['tags'] = df['tags'].apply(lambda x: ', '.join([item['name'] for item in x]))
df['owners'] = df['owners'].apply(lambda x: x[0] if x else '')

df.head()

,dag_id,start_date,end_date,run_type,state,start_hour,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,earthquake_data_upstream,22-10-2024 14:03:39,22-10-2024 14:04:11,manual,success,14,0 days 00:00:31.508536,Upstream EarthQuake Data,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"
1,teste_falha,22-10-2024 14:03:38,22-10-2024 14:10:19,manual,failed,14,0 days 00:06:40.432990,Teste de Falha,True,False,False,None,Tester,"test, ativo",None,"Never, external triggers only"
2,teste_execucao,22-10-2024 14:03:38,22-10-2024 14:07:58,manual,success,14,0 days 00:04:19.294494,Teste de Execucao,True,False,False,None,Tester,"test, ativo",None,"Never, external triggers only"
3,processa_dados_EarthQuake,22-10-2024 14:03:39,22-10-2024 14:04:07,manual,success,14,0 days 00:00:27.984439,Data Count,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"
4,earthquake_data_upstream,22-10-2024 14:09:47,22-10-2024 14:10:25,manual,success,14,0 days 00:00:38.089895,Upstream EarthQuake Data,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"


In [31]:
df.to_csv('./dados_modelo.csv', index=False, header=True)

# Dados extraidos

In [33]:
df = pd.read_csv('./dados_modelo.csv', sep=';')
df.head()

,dag_id,start_date,end_date,run_type,state,start_hour,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,earthquake_data_upstream,22/10/2024 14:03,22/10/2024 14:04,manual,success,14,0 days 00:00:31.508536,Upstream EarthQuake Data,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"
1,teste_falha,22/10/2024 14:03,22/10/2024 14:10,manual,failed,14,0 days 00:06:40.432990,Teste de Falha,True,False,False,NaN,Tester,"test, ativo",NaN,"Never, external triggers only"
2,teste_execucao,22/10/2024 14:03,22/10/2024 14:07,manual,success,14,0 days 00:04:19.294494,Teste de Execucao,True,False,False,NaN,Tester,"test, ativo",NaN,"Never, external triggers only"
3,processa_dados_EarthQuake,22/10/2024 14:03,22/10/2024 14:04,manual,success,14,0 days 00:00:27.984439,Data Count,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"
4,earthquake_data_upstream,22/10/2024 14:09,22/10/2024 14:10,manual,success,14,0 days 00:00:38.089895,Upstream EarthQuake Data,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"


In [5]:
df['hora'] = df['start_date'].dt.hour
df['hora_arredondada'] = df['data_hora'].dt.round('H')

AttributeError: Can only use .dt accessor with datetimelike values